In [1]:
# tracks={}
# for no in ['A','B','C','D']:
#     tracks[no] = pd.read_csv("dataset/track_" + no + ".csv", parse_dates=["date"])

In [2]:
import pandas_profiling as pdp

In [3]:
summarys4 = {}
for no in ['A', 'B', 'C', 'D']:
    summarys4[no] = pd.read_csv(f'summary4_{no}.csv', index_col=0)

In [4]:
for no in ['A', 'B', 'C', 'D']:
    summarys4[no].loc[summarys4[no]['傾き'] == -100, '傾き'] = np.median(summarys4[no].loc[summarys4[no]['傾き'].notnull(), '傾き'])
    summarys4[no].loc[summarys4[no]['傾き'].isnull(), '傾き'] = np.median(summarys4[no].loc[summarys4[no]['傾き'].notnull(), '傾き'])
    summarys4[no].loc[summarys4[no]['切片'].isnull(), '切片'] = np.median(summarys4[no].loc[summarys4[no]['切片'].notnull(), '切片'])

In [5]:
from fbprophet import Prophet

In [6]:
summarys4['A'].dropna().loc[:100]

,路線,キロ程,傾き,切片,標本標準偏差,測定回数,データ数,バラスト,ロングレール,マクラギ種別,橋りょう,踏切,通トン,曲線半径,フラグ
0,A,10000.0,-0.002443,0.570,3.795202,261.0,63.0,1,1,3,0,0,13.607,0,1
1,A,10001.0,-0.002372,2.680,3.165688,260.0,53.0,1,1,3,0,0,13.607,0,1
2,A,10002.0,-0.002152,5.005,3.497406,259.0,43.0,1,1,3,0,0,13.607,0,1
3,A,10003.0,-0.003085,5.880,4.083893,259.0,44.0,1,1,3,0,0,13.607,0,1
4,A,10004.0,-0.001037,5.035,3.805207,261.0,61.0,1,1,1,0,0,13.607,0,1
5,A,10005.0,0.000754,1.645,2.261965,261.0,58.0,1,1,1,0,0,13.607,0,1
6,A,10006.0,0.001475,-3.100,2.588153,257.0,57.0,1,1,1,0,0,13.607,0,1
7,A,10007.0,0.001710,-7.240,4.033433,255.0,61.0,1,1,1,0,0,13.607,0,1
8,A,10008.0,0.002739,-8.850,4.599720,251.0,61.0,1,1,1,0,0,13.607,0,1
9,A,10009.0,0.000384,-7.410,3.482811,251.0,46.0,1,1,1,0,0,13.607,0,1


In [ ]:
data = pd.DataFrame(columns=['ds', 'y'])
summary = summarys4['A'].dropna().loc[:10000, ['キロ程', '切片']]
# summary = summarys4['A'].dropna()

data['ds'] = summary['キロ程']
data['y'] = summary['切片']
model = Prophet(growth='linear', daily_seasonality=False)
model.add_seasonality(name='monthly', period=30.5, fourier_order=5)
model.fit(data)

INFO:fbprophet.forecaster:Disabling yearly seasonality. Run prophet with yearly_seasonality=True to override this.
INFO:fbprophet.forecaster:Disabling weekly seasonality. Run prophet with weekly_seasonality=True to override this.


In [ ]:
future = model.make_future_dataframe(periods=90)
forecast = model.predict(future)
model.plot(forecast)
plt.show()

In [ ]:
model.plot_components(forecast)
plt.show()

In [ ]:
# add change rates to changepoints
df_changepoints = data.loc[model.changepoints.index]
df_changepoints['delta'] = model.params['delta'].ravel()

# get changepoints
df_changepoints['ds'] = df_changepoints['ds'].astype(str)
df_selection = df_changepoints[df_changepoints['delta'] != 0]
date_changepoints = df_selection['ds'].astype('datetime64[ns]').reset_index(drop=True)

# plot
sns.set(style='whitegrid')
ax = sns.factorplot(x='ds', y='delta', data=df_changepoints, kind='bar', color='royalblue', size=4, aspect=2)
ax.set_xticklabels(rotation=90)

In [ ]:
for no in ['A', 'B', 'C', 'D']:
    summarys[no].to_csv(f'summary6_{no}.csv')

In [ ]:
import seaborn as sns

In [ ]:
%%time
submit = pd.DataFrame()
for no in ['A', 'B', 'C', 'D']:
    track = tracks[no]
    summary = summarys[no]
    summary = summary.dropna()

    sub = pd.DataFrame()
    sub['キロ程'] = track['キロ程'].unique()
    sub['路線'] = no

    kilos = summary['キロ程'].values
    inds = kilos.astype(int) - 10000

    sub.loc[inds, '傾き'] = summary['傾き'].values
    sub.loc[inds, '切片'] = summary['切片'].values
    
    submit = pd.concat([submit, sub], axis=0)
submit = submit.reset_index(drop=True)

In [ ]:
pdp.ProfileReport(submit)

In [ ]:
p = []
last_median_day = 30
for no in ['A', 'B', 'C', 'D']:
    tmp = submit[submit['路線'] == no]
    for i in range(91):
        day = tmp['切片'].values + tmp['傾き'].values*(i+last_median_day//2)
        p.extend(day)

submision = pd.DataFrame(p)
print(sum(submision.isnull()))
submision = submision.fillna(0)
submision = submision.apply(lambda x:x.round(2)) #サイズが大きいため予測値を桁落とし

In [ ]:
submision.max()

In [ ]:
submision.min()

In [ ]:
submision.loc[submision[0] > 16, 0] = 16
submision.loc[submision[0] < -16, 0] = -16

In [ ]:
submision

In [ ]:
import random
import datetime

In [ ]:
def get_day(dates):
    day = [(date - datetime.datetime(2017, 4, 1)).days for date in dates]
    return day

In [ ]:
random.seed(0)

fig, axes = plt.subplots(20, 1, figsize=(15, 40), sharex=True)

track = tracks['A']
kilos = track['キロ程'].unique()
for i, kilo in enumerate(random.sample(list(kilos), 20)):
    data = track.loc[track['キロ程'] == kilo, ['date', '高低左']]
    data = data.dropna(subset=['高低左'])
    data = data.reset_index(drop=True)
    data['day'] = get_day(data['date'])
    data = data.drop(['date'], axis=1)
    axes[i].plot(data['day'], data['高低左'], marker='.', linewidth=0, c='c')

    kilo_id = kilo - 10000
    hoge = [kilo_id + i*len(kilos) for i in range(91)]
    piyo = submision.loc[hoge, 0]
    axes[i].plot(range(365, 365 + 91), piyo, c='g')

    axes[i].set_title(f'kilo: {kilo},  kilo_id: {kilo_id}')

In [ ]:
random.seed(0)

fig, axes = plt.subplots(20, 1, figsize=(15, 40), sharex=True)

track = tracks['B']
kilos = track['キロ程'].unique()
for i, kilo in enumerate(random.sample(list(kilos), 20)):
    data = track.loc[track['キロ程'] == kilo, ['date', '高低左']]
    data = data.dropna(subset=['高低左'])
    data = data.reset_index(drop=True)
    data['day'] = get_day(data['date'])
    data = data.drop(['date'], axis=1)
    axes[i].plot(data['day'], data['高低左'], marker='.', linewidth=0, c='c')

    kilo_id = kilo - 10000
    hoge = [kilo_id + i*len(kilos) for i in range(91)]
    hoge = np.array(hoge) + len(tracks['A']['キロ程'].unique())*91
    piyo = submision.loc[hoge, 0]
    axes[i].plot(range(365, 365 + 91), piyo, c='g')

    axes[i].set_title(f'kilo: {kilo},  kilo_id: {kilo_id}')

In [ ]:
random.seed(0)

fig, axes = plt.subplots(20, 1, figsize=(15, 40), sharex=True)

track = tracks['C']
kilos = track['キロ程'].unique()
for i, kilo in enumerate(random.sample(list(kilos), 20)):
    data = track.loc[track['キロ程'] == kilo, ['date', '高低左']]
    data = data.dropna(subset=['高低左'])
    data = data.reset_index(drop=True)
    data['day'] = get_day(data['date'])
    data = data.drop(['date'], axis=1)
    axes[i].plot(data['day'], data['高低左'], marker='.', linewidth=0, c='c')

    kilo_id = kilo - 10000
    hoge = [kilo_id + i*len(kilos) for i in range(91)]
    hoge = np.array(hoge) + len(tracks['A']['キロ程'].unique())*91 + len(tracks['B']['キロ程'].unique())*91
    piyo = submision.loc[hoge, 0]
    axes[i].plot(range(365, 365 + 91), piyo, c='g')

    axes[i].set_title(f'kilo: {kilo},  kilo_id: {kilo_id}')

In [ ]:
random.seed(0)

fig, axes = plt.subplots(20, 1, figsize=(15, 40), sharex=True)

track = tracks['D']
kilos = track['キロ程'].unique()
for i, kilo in enumerate(random.sample(list(kilos), 20)):
    data = track.loc[track['キロ程'] == kilo, ['date', '高低左']]
    data = data.dropna(subset=['高低左'])
    data = data.reset_index(drop=True)
    data['day'] = get_day(data['date'])
    data = data.drop(['date'], axis=1)
    axes[i].plot(data['day'], data['高低左'], marker='.', linewidth=0, c='c')

    kilo_id = kilo - 10000
    hoge = [kilo_id + i*len(kilos) for i in range(91)]
    hoge = np.array(hoge) + len(tracks['A']['キロ程'].unique())*91 + len(tracks['B']['キロ程'].unique())*91 + len(tracks['C']['キロ程'].unique())*91
    piyo = submision.loc[hoge, 0]
    axes[i].plot(range(365, 365 + 91), piyo, c='g')

    axes[i].set_title(f'kilo: {kilo},  kilo_id: {kilo_id}')

In [ ]:
submision.to_csv("submit6.csv", header=None)